In [1]:
import requests
import os,sys,glob,shutil
import zipfile
import earthaccess

## Step 1: Define a data directory where all the auxillary data is saved
* It is convinient to save all data at a one location, in different folders
* By default, the notebook will try to assign that to a data folder in the Home directory
* If you have a preference, please provide that below

In [7]:
prefered_data_dir = '/nobackup/sbhusha1/reference_data/' #uncomment this and provide directory of your choice
#prefered_data_dir = None # Comment this out if uncommenting the above
if prefered_data_dir == None:
    data_directory = os.path.expanduser("~/data/")
else:
    data_directory = prefered_data_dir

print(f"Data will be saved in {data_directory}")

# Create the directory
if not os.path.exists(data_directory):
    os.makedirs(data_directory)
# save this to bash export
! export DATADIR=$data_directory

Data will be saved in /nobackup/sbhusha1/reference_data/


### Setting reference data folder
```{admonition}Tip
**Please add the export DATADIR line to your .bashrc file for seamless access by all command line scripts in the future.**

**For my case, it looks like: export DATADIR=/nobackupp16/sbhusha1/reference_data/**
```
 

## Step 2: Download RGI Shapefiles
* We will fetch RGI 15 shapefiles for our study area


In [3]:
def download_fn(url,outpath):
    if os.path.exists(outpath):
        print(f"file {outpath} already downloaded")
    else:
        response = requests.get(url)
        #Check for 200
        if response.ok:
            print ('OK!')
        else:
            print ('Query failed')
            sys.exit()
        #Write to disk
        open(outpath, 'wb').write(response.content)
        print(f"file {outpath} saved")

In [8]:
rgi_zip_url =  "http://www.glims.org/RGI/rgi60_files/15_rgi60_SouthAsiaEast.zip"
rgi_shp_fn = os.path.join(data_directory,"15_rgi60_SouthAsiaEast.zip")
download_fn(rgi_zip_url,rgi_shp_fn) 

OK!
file /nobackup/sbhusha1/reference_data/15_rgi60_SouthAsiaEast.zip saved


In [9]:
rgi_directory = os.path.join(data_directory,'rgi60/regions/rgi15/')
with zipfile.ZipFile(rgi_shp_fn, 'r') as zipObj:
   # Extract all the contents of zip file in different directory
   zipObj.extractall(rgi_directory)

## Step 3: Download ice thickness files from Farinotti et al. 2019
* We will again download files for the region 15 for concensus estimates
* We will also download files for individual models for uncertainty estimates 


In [10]:
ice_thickness_farinotti_url = 'https://www.research-collection.ethz.ch/bitstream/handle/20.500.11850/315707/composite_thickness_RGI60-15.zip?sequence=17&isAllowed=y'
ice_thickness_farinotti_zip = os.path.join(data_directory,'composite_thickness_RGI60-15.zip')

download_fn(ice_thickness_farinotti_url,ice_thickness_farinotti_zip)

OK!
file /nobackup/sbhusha1/reference_data/composite_thickness_RGI60-15.zip saved


In [12]:
## The file above contains data of ice thickness for each glacier in the region as an individual tiff file, 
#which will take a long time to unzip everything.
# so will selectively unzip files for only the six glaciers in the study
rgi_id_list = ['RGI60-15.03473','RGI60-15.03733','RGI60-15.03734','RGI60-15.03743','RGI60-15.04121','RGI60-15.04045']
ice_thickness_farinotti_dir = os.path.join(data_directory,'farinotti_2019_ice_thickness/')
with zipfile.ZipFile(ice_thickness_farinotti_zip, 'r') as zipObj:
    for rgi in rgi_id_list:
      thickness_fn = f'RGI60-15/{rgi}_thickness.tif'
      zipObj.extract(thickness_fn, 
                     path=ice_thickness_farinotti_dir)
      


## Step 4: Individual model output download

In [8]:
model1_url = 'https://www.research-collection.ethz.ch/bitstream/handle/20.500.11850/315707/results_model_1.zip?sequence=22&isAllowed=y'
model2_url = 'https://www.research-collection.ethz.ch/bitstream/handle/20.500.11850/315707/results_model_2.zip?sequence=23&isAllowed=y'
model3_url = 'https://www.research-collection.ethz.ch/bitstream/handle/20.500.11850/315707/results_model_3.zip?sequence=24&isAllowed=y'
model4_url = 'https://www.research-collection.ethz.ch/bitstream/handle/20.500.11850/315707/results_model_4.zip?sequence=25&isAllowed=y'
#model5 is not available for the region
model1_zip = os.path.join(data_directory,'results_model_1.zip')
model2_zip = os.path.join(data_directory,'results_model_2.zip')
model3_zip = os.path.join(data_directory,'results_model_3.zip')
model4_zip = os.path.join(data_directory,'results_model_4.zip')


In [9]:
download_fn(model1_url,model1_zip)
download_fn(model2_url,model2_zip)
download_fn(model3_url,model3_zip)
download_fn(model4_url,model4_zip)

OK!
file /nobackup/sbhusha1/reference_data/results_model_1.zip saved
OK!
file /nobackup/sbhusha1/reference_data/results_model_2.zip saved
OK!
file /nobackup/sbhusha1/reference_data/results_model_3.zip saved
OK!
file /nobackup/sbhusha1/reference_data/results_model_4.zip saved


In [13]:
## The file above contains data of ice thickness for each glacier in the region as an individual tiff file, 
#which will take a long time to unzip everything.
# so will selectively unzip files for only the six glaciers in the study
rgi_id_list = ['RGI60-15.03473','RGI60-15.03733','RGI60-15.03734','RGI60-15.03743','RGI60-15.04121','RGI60-15.04045']
ice_thickness_farinotti_dir = os.path.join(data_directory,'farinotti_2019_ice_thickness/model1/')
for idx,zippedfile in enumerate([model1_zip,model2_zip,model3_zip,model4_zip]):
    ice_thickness_farinotti_dir = os.path.join(data_directory,f'farinotti_2019_ice_thickness/model{idx+1}/')
    with zipfile.ZipFile(zippedfile, 'r') as zipObj:
        for rgi in rgi_id_list:
          thickness_fn = f'RGI60-15/thickness_{rgi}.tif'
          zipObj.extract(thickness_fn, 
                         path=ice_thickness_farinotti_dir)

## Step 5: Download debris thickness files from Rounce et al. 2020 study
* We will download files from NSIDC website using the earthacess library
* This requires us to have a NASA EarthData login

In [14]:
auth = earthaccess.login()

Enter your Earthdata Login username:  sbaglapl@uw.edu
Enter your Earthdata password:  ········


In [15]:
bounding_box = [84.44955936858184,26.811496290752743,87.98311172730928,29.272110001382174] #of our study site
Query = earthaccess.granule_query().short_name('HMA_DTE').version("1").bounding_box(*bounding_box)
Query.hits()

916

In [16]:
debris_thickness_directory = os.path.join(data_directory,'debris_thickness/')
granules = Query.get_all()
files = earthaccess.download(granules, debris_thickness_directory)

 Getting 916 granules, approx download size: 0.02 GB


QUEUEING TASKS | :   0%|          | 0/3820 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/3820 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/3820 [00:00<?, ?it/s]

## Step 6: Download Millan ice thickness files
* We will download files from [Theia website](https://www.sedoo.fr/theia-publication-products/?uuid=55acbdd5-3982-4eac-89b2-46703557938c)
* We need to create download links for the area we want, for this study it is Central and South Asia.

In [14]:

## Create data link from here: https://www.sedoo.fr/theia-publication-products/?uuid=55acbdd5-3982-4eac-89b2-46703557938c 
ice_thickness_millan_url = 'https://services-theia.sedoo.fr/glaciers/data/ige/v1_0/download/35021cd3-2eba-418c-9ead-829bfa24bf72'
ice_thickness_millan_zip = os.path.join(data_directory,'RGI-13-15.zip')

download_fn(ice_thickness_millan_url,ice_thickness_millan_zip)

ConnectTimeout: HTTPSConnectionPool(host='services-theia.sedoo.fr', port=443): Max retries exceeded with url: /glaciers/data/ige/v1_0/download/35021cd3-2eba-418c-9ead-829bfa24bf72 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7ff2ded07dd0>, 'Connection to services-theia.sedoo.fr timed out. (connect timeout=None)'))

In [14]:
millan_thickness_directory = os.path.join(data_directory,'millan_thickness/')
with zipfile.ZipFile(ice_thickness_millan_zip, 'r') as zipObj:
   # Extract all the contents of zip file in different directory
   zipObj.extractall(millan_thickness_directory)

OSError: [Errno 28] No space left on device

In [ ]:
surface_velocity_millan_url = 'https://services-theia.sedoo.fr/glaciers/data/ige/v1_0/download/4100ce38-31e2-4c11-9be4-f14827728faa'
surface_velocity_millan_zip = os.path.join(data_directory,'RGI-13-15_velocity.zip')

if os.path.exists(surface_velocity_millan_zip):
    print(f"file {surface_velocity_millan_zip} already downloaded")
else:
    response = requests.get(surface_velocity_millan_url)
    #Check for 200
    if response.ok:
        print ('OK!')
    else:
        print ('Query failed')
        sys.exit()
    #Write to disk
    open(surface_velocity_millan_zip, 'wb').write(response.content)
    print(f"file {surface_velocity_millan_zip} saved")

In [ ]:
millan_velocity_directory = os.path.join(data_directory,'millan_velocity/')
with zipfile.ZipFile(surface_velocity_millan_zip, 'r') as zipObj:
   # Extract all the contents of zip file in different directory
   zipObj.extractall(millan_velocity_directory)